# **EDA with SQL**
---

In [4]:
%load_ext sql

In [15]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()    

In [16]:
%sql sqlite:///my_data1.db

In [17]:
import pandas as pd 

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi") 

101

In [31]:
df.columns

Index(['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload',
       'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome',
       'Landing_Outcome'],
      dtype='object')

In [18]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

   sqlite:///my_data.db
 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [21]:
%sql select distinct Launch_Site from SPACEXTABLE

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [29]:
%%sql select * from SPACEXTABLE where Launch_Site like 'CCA%'
limit 5

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [35]:
%sql select SUM(PAYLOAD_MASS__KG_) from SPACEXTABLE

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


SUM(PAYLOAD_MASS__KG_)
619967


In [36]:
%sql select AVG(PAYLOAD_MASS__KG_) from SPACEXTABLE

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


AVG(PAYLOAD_MASS__KG_)
6138.287128712871


In [38]:
%sql select AVG(PAYLOAD_MASS__KG_) from SPACEXTABLE where Booster_Version like 'F9 v1.0%'

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


AVG(PAYLOAD_MASS__KG_)
340.4


In [43]:
%sql select * from SPACEXTABLE where date = (select min(date) from SPACEXTABLE)

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)


In [44]:
# Tabmién se puede emplear esto
%sql select * from SPACEXTABLE order by date limit 1

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)


In [50]:
%sql Select Payload from SPACEXTABLE where Landing_Outcome = 'Success (drone ship)'  and PAYLOAD_MASS__KG_ > 4000 and PAYLOAD_MASS__KG_ < 6000

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


Payload
JCSAT-14
JCSAT-16
SES-10
SES-11 / EchoStar 105


In [54]:
%sql select distinct(Mission_Outcome) from SPACEXTABLE

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


Mission_Outcome
Success
Failure (in flight)
Success (payload status unclear)
Success


In [58]:
%%sql select Mission_Outcome, count(*) as Outcome_Count
from SPACEXTABLE
where Mission_Outcome in ('Success', 'Failure (in flight)')
Group by Mission_Outcome

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


Mission_Outcome,Outcome_Count
Failure (in flight),1
Success,98


In [60]:
%sql select max(PAYLOAD_MASS__KG_) from SPACEXTABLE

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


max(PAYLOAD_MASS__KG_)
15600


In [64]:
%%sql
SELECT Booster_Version
from SPACEXTABLE
where PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) from SPACEXTABLE)

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [72]:
%%sql
select substr(Date, 6,2), landing_outcome, booster_version, launch_site 
from SPACEXTABLE
where substr(Date, 0, 5) = "2015"
and landing_outcome = 'Failure (drone ship)'

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


"substr(Date, 6,2)",Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [75]:
%%sql
SELECT landing_outcome, count(*) as outcome_count
from SPACEXTABLE
where Date Between '2010-06-04' and '2017-03-20'
group by landing_outcome
order by outcome_count Desc

   sqlite:///my_data.db
 * sqlite:///my_data1.db
Done.


Landing_Outcome,outcome_count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
